In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import sys
sys.path.append('..')


In [ ]:
import pandas as pd
import autograd.numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image

from utils.models import BNN, BNN_LV
from utils.functions import gaussian, log_gaussian
from utils.training import HMC


## Set up Data and PDFs


In [ ]:
# Use the test data from HW8
data = pd.read_csv('../data/HW8_data.csv')
X_train = np.array(data['x']).reshape(12,1)
y_train = np.array(data['y']).reshape(12,1)


# Set up the prior, likelihood and posterior
def log_prior(W, mu, sigma):
    """ Generate the prior PDF """
    return np.sum(log_gaussian(x=W, mu=mu, sigma=sigma), axis=-1)


# Set up the prior, likelihood and posterior
def log_latent_prior(z, mu, gamma):
    """ Generate the prior PDF """
    return np.sum(np.sum(log_gaussian(x=z, mu=mu, sigma=gamma), axis=-1), axis=-1)


def log_likelihood(W, X, Y, mu, sigma):
    """ Generate the likelihood PDF """
    llh = np.sum(log_gaussian(x=Y, mu=mu, sigma=sigma), axis=0)
    return llh


def create_log_posterior(X, Y, p_mu, p_sigma, l_sigma, nn, gamma=False, latent=False):
    """ Wrapper to create an initialized posterior PDF """
    if latent == False:
        def log_posterior(W, X=X, Y=Y, p_mu=p_mu, p_sigma=p_sigma, l_sigma=l_sigma, nn=nn):
            """ Generate the posterior PDF """
            mu_l = nn.forward(X, weights=W)
            log_p = log_prior(W=W, mu=p_mu, sigma=p_sigma)
            log_l = log_likelihood(W=W, X=X, Y=Y, mu=mu_l, sigma=l_sigma)
            llh = log_p + log_l
            return llh 

    elif latent == True:
        def log_posterior(W, X=X, Y=Y, p_mu=p_mu, p_sigma=p_sigma, l_sigma=l_sigma, nn=nn, gamma=gamma):
            """ Generate the posterior PDF """
            mu_l = nn.forward(X, weights=W)
            z = nn.last_input_noise
            log_p = log_prior(W=W, mu=p_mu, sigma=p_sigma)
            log_zp = log_latent_prior(z=z, mu=0, gamma=gamma)
            log_l = log_likelihood(W=W, X=X, Y=Y, mu=mu_l, sigma=l_sigma)
            llh = log_p + log_l + log_zp
            return llh 
    else:
        raise ValueError("Error: latent must be one of [True, False]")

    return log_posterior


## Train a BNN+LV using HMC


In [ ]:
# Parameters
gamma = 1
sigma = 1

architecture = {
    'input_n':1, 
    'output_n':1, 
    'hidden_layers':[5],
    'biases' : [1,1],
    'activations' : ['relu', 'linear'],
    'gamma':[gamma],
    'sigma':[sigma],
}
bnn_lv = BNN_LV(architecture=architecture)

bnn_lv.fit(X_train, y_train, step_size=0.01, max_iteration=5000, check_point=500, regularization_coef=None)


## Define W&B callbacks to make plots


In [ ]:
# Import helpers for building callbacks:
from utils.training import build_wb_callback_postpred
from utils.training import build_wb_callback_plotfunc

# Build a callback that produces a scatter plot using W&B built-in functions:
wb_callback_postpred = build_wb_callback_postpred(model=bnn_lv, x_data=X_train)

# # Import plotting function:
# from utils.plot import plot_posterior_predictive

# # Build a callback from a user-defined plotting function:
# wb_callback_plotfunc = build_wb_callback_plotfunc(
#     filename='hmc_plot.png', plot_func=plot_posterior_predictive,
#     model=bnn_lv, x_data=X_train, mode='fill',
#     figsize=(14,7), real_x=None, real_y=None,
# )


## Run HMC with W&B logging:


In [ ]:
# Define W&B settings:
wb_settings = {
    'entity' : 'gpestre',
    'project' : 'am207',
    'group' : 'demos',
    'name' : 'callback_demo',
    'notes' : 'Small test case',
    'progress' : 10,
    'base_path' : '../data/',
    'filename' : 'temp_hmc_state.json',
    'archive' : {  # Manually archive info about network and priors.
        'architecture' : architecture,
    },
    'callback' : wb_callback_postpred,
    #'callback' : [wb_callback_postpred, wb_callback_plotfunc],
}


In [ ]:
# Build the posterior:
log_posterior_bnn_lv = create_log_posterior(X_train, y_train, 0, 5, 0.25, bnn_lv, gamma=gamma, latent=True)

# Use weights found from backprop as initial point for HMC:
mle_lv_weights = bnn_lv.get_weights()

# Sample from HMC:
hmc = HMC(
    log_target_func=log_posterior_bnn_lv, position_init=mle_lv_weights,
    total_samples=100, burn_in=0.5, thinning_factor=1, 
    leapfrog_steps=20, step_size=1e-3, mass=1.0, random_seed=207,
    progress=5, wb_settings=wb_settings,
)
